#### Dependencies:

In [1]:

from kcat_prediction import *

Before using kcat prediction for the first time, we have to download the ESM-1b model:

In [2]:
import esm
model, alphabet = torch.hub.load("facebookresearch/esm:v0.4.0", "esm1b_t33_650M_UR50S")

#model, alphabet = torch.hub.load("facebookresearch/esm:v0.4.0", "esm1v_t33_650M_UR90S")



Using cache found in /n/home00/stephentsou/.cache/torch/hub/facebookresearch_esm_v0.4.0


Example for calculating kcat prediction. You can enter multiple enyzmes and substrates. Enzyme 1 will be paired with substrates 1/products 1, enzyme 2 will be paired with substrates 2/products 2 and so on...

Enter enzymes as amino acid sequences and substrates/products either as SMILES strings, KEGG Compound IDs, or InChI strings. If the reaction has multiple substrates or products, separate them by using a semicolon (";)

In [3]:
#substrates = ["InChI=1S/C7H5NO4/c9-8(10)5-1-2-6-7(3-5)12-4-11-6/h1-3H,4H2;InChI=1S/H2O2/c1-2/h1-2H",]
##tetrahydrofolate SMILES = C1[C@@H](NC2=C(N1)N=C(NC2=O)N)CNC3=CC=C(C=C3)C(=O)N[C@@H](CCC(=O)O)C(=O)O



substrates = ["InChI=1S/C19H23N7O6/c20-19-25-15-14(17(30)26-19)23-11(8-22-15)7-21-10-3-1-9(2-4-10)16(29)24-12(18(31)32)5-6-13(27)28/h1-4,11-12,21,23H,5-8H2,(H,24,29)(H,27,28)(H,31,32)(H4,20,22,25,26,30)/t11-,12-/m0/s1;InChI=1S/C21H27N7O14P2/c22-17-12-19(25-7-24-17)28(8-26-12)21-16(32)14(30)11(41-21)6-39-44(36,37)42-43(34,35)38-5-10-13(29)15(31)20(40-10)27-3-1-2-9(4-27)18(23)33/h1-4,7-8,10-11,13-16,20-21,29-32H,5-6H2,(H5-,22,23,24,25,33,34,35,36,37)/p+1/t10-,11-,13-,14-,15-,16-,20-,21-/m1/s1",]



#products = ["InChI=1S/C6H5NO4/c8-5-2-1-4(7(10)11)3-6(5)9/h1-3,8-9H;InChI=1S/CH2O2/c2-1-3/h1H,(H,2,3);InChI=1S/H2O/h1H2"]

#dihydrofolate acid, NADPH
products = ["InChI=1S/C19H21N7O6/c20-19-25-15-14(17(30)26-19)23-11(8-22-15)7-21-10-3-1-9(2-4-10)16(29)24-12(18(31)32)5-6-13(27)28/h1-4,12,21H,5-8H2,(H,24,29)(H,27,28)(H,31,32)(H4,20,22,25,26,30)/t12-/m0/s1;InChI=1S/C21H30N7O17P3/c22-17-12-19(25-7-24-17)28(8-26-12)21-16(44-46(33,34)35)14(30)11(43-21)6-41-48(38,39)45-47(36,37)40-5-10-13(29)15(31)20(42-10)27-3-1-2-9(4-27)18(23)32/h1,3-4,7-8,10-11,13-16,20-21,29-31H,2,5-6H2,(H2,23,32)(H,36,37)(H,38,39)(H2,22,24,25)(H2,33,34,35)/t10-,11-,13-,14-,15-,16-,20-,21-/m1/s1;InChI=1S/p+1"]

#H+
#;1S/p+1


#enzymes = ["MKYFPLFPTLVFAARVVAFPAYASLAGLSQQELDAIIPTLEAREPGLPPGPLENSSAKLVNDEAHPWKPLRPGDIRGPCPGLNTLASHGYLPRNGVATPVQIINAVQEGLNFDNQAAVFATYAAHLVDGNLITDLLSIGRKTRLTGPDPPPPASVGGLNEHGTFEGDASMTRGDAFFGNNHDFNETLFEQLVDYSNRFGGGKYNLTVAGELRFKRIQDSIATNPNFSFVDFRFFTAYGETTFPANLFVDGRRDDGQLDMDAARSFFQFSRMPDDFFRAPSPRSGTGVEVVIQAHPMQPGRNVGKINSYTVDPTSSDFSTPCLMYEKFVNITVKSLYPNPTVQLRKALNTNLDFFFQGVAAGCTQVFPYGRD"]

#DHFR
#enzymes = ["MVGSLNCIVAVSQNMGIGKNGDLPWPPLRNEFRYFQRMTTTSSVEGKQNLVIMGKKTWFSIPEKNRPLKGRINLVLSRELKEPPQGAHFLSRSLDDALKLTEQPELANKVDMVWIVGGSSVYKEAMNHPGHLKLFVTRIMQDFESDTFFPEIDLEKYKLLPEYPGVLSDVQEEKGIKYKFEVYEKND"]

#1
#enzymes = ["MISFIFAMDANRLIGKDNDLPWHLPNDLAYFKKITSGHSIIMGRKTFESIGRPLPNRKNIVVTSAPDSEFQGCTVVSSLKDVLDICSGPEECFVIGGAQLYTDLFPYADRLYMTKIHHEFEGDRHFPEFDESNWKLVSSEQGTKDEKNPYDYEFLMYEKKNSSKVGGF"]
#2
#enzymes = ["MTFSLIVATTLNSVIGKDNQMPWHLPADLAWFRQNTTGKPVIMGRKTFESIGRPLPKRTNIVLSRQPFKHEGVVWKNSLESAVNFVRDFDEIMLIGGGELFKQYLPKADKLYLTQIQTELDGDTFFPQLNWEEWKIEFDEYHKADEQNRYDCRSLILTRK"]
#3
#enzymes = ["MTFSLIVAMTKNHVIGKDNQMPWHLPADLAWFRQNTTGKPVIMGRKTFESIGRPLPKRTNLVLSRQPYLHEGVIWQNSLESAVDFVKDSPEILLIGGGELFKQYLHKADKLYLTEIQTELEGDTFFPEIVWQDWQIEYEAWHQADEKNPYDCRFLILSRNTKNE"]
#4
#enzymes = ["MIIGIWAEDEAGLIGEADKMPWSLPAEQQHFKETTMNQVILMGRKTFEGMNKRVLPGRISIILTRDETYQSDNEKVLIMHSPKEVLDWYHKQNKDLFITGGAEILALFESELELLYRTVVHEKFKGDTYFPSTFDFGRFKLVSEKFHDKDERNSYTFTIKKYEKVKQP"]
#5
#enzymes = ["MLSIIAALNENYVIGNENKLIWHISDDLKRFKKLTTGKTIIMGRKTFESLPGVLPNRKHIVITKNLNYSRENVSIVHSIDEIIELKDTTEENFIIGGGEIYRALIPYCSILYLTKVHSNQTGDAFFPKFDENDYSIIATEKHGDYDFVTFKRR"]
#6
#enzymes = ["MIIFVWAQDRAGNIGKDNKMPWHLPGDLQFFKKTTTGKTLVMGRKTYESLGKALPNRKTIVLTRDNELQLDDAEILHSRDEVLALAETGEPIYVVGGAEIYRLFMDVADKLIVTKIDAEFDADTAFPEVDWENFSEVAKEPHEKDEKNKYNYTFYTYERN"]
#7
#enzymes = ["MPMPATPLLSLVVAYSTNGVIGRDNALPWKLPGDLAHFKRTTLGHPIIMGRNTWESLGRPLPGRTNIVITRNPAYGAAGAVVVGSLDAAIQACGDAAVACVIGGAQIYAQALPLAQRVVATEVHAEIEGDAFFPPLPAGQWRETQRAAQPAENGLRYDFVEYERVAG"]
#8
#enzymes = ["MIISLIAALAADRVIGMENAMPWHLPADLAWFKRNTLNKPVIMGRKTFESIGRPLPGRLNIVISSQPGTDERVTWAASIEEALAFAGNAEEVMVMGGGRVYKQFLDRANRMYLTHIDAEVGGDTHFPDYEPDEWESVFSEFHDADEANSHSYCFEILERR"]
#9
#enzymes = ["MTRPPDISLIVAMARNRVIGADNAMPWHLSADLKRFRALTLGKPVLMGRKTHESIGRVLPGRQNIILTRSPDFHAPGCTIVHDLGELATVCSGAPELMVIGGSAVYEALLPRARRLYLTLIDHDYPGDTYFPEFAAADWREISREDVRNDPDFPWPYSYLVLERASPEARCQ"]
#10
#enzymes = ["MSGTGQPRVSVIAALAKNRVIGIENRLPWRLPEDLAHFKALTLDHPVLMGRKTFESLGRPLPRRTNVVITRNADYRPDGCLVAASMPAAIALCQGADEIFFIGGAELYAQAIPLADRLYLTEVDAEPEGDAWFPEFDRAAFREISRASHVGEKGDVLRFDFVVYERR"]
#11
#enzymes = ["MAQLTLIVAYARNRVIGRDNTLPWRLPGDLAHFKRSTLGHPIVMGRNTWESLGRPLPGRKNIVVSRNPDYRAEGAIVVPDLQSALKMAEADEVFVIGGAQIYSQALPLATRIIATEIQANVAGDAYFPPLAQTEWRETSRQPQPEENGYHYDFVVYERNR"]
#12
#enzymes = ["MTKIKMVWAEDRQHAIGKDGGIPWHMPDDLKLFRDETVNTLMIMGRPTWLSIGRPLPKRTTVVMTRQEDWTPSYPEVKVIHSIEEAKNLIAKEERDITIAGGAAVYREFMPYATDLVITRVDGVIDGDTFVDEVDLTQFQLKSSEPHAKDDNHDYAFVVERYERI"]
#13
#enzymes = ["MMISMIAAMAHDRVIGLDNQMPWHLPADLAHFKRVTLGKPVLMGRKTFESIGRPLPGRRNLVISRNPDYRADGVEVIDSVDAALALLAGSDVAELMVIGGGHLYGQLLPRADRLYLTRIDLAVEGDTRFPAFDEGDWSCIERESHQPDEKNPHSYCFETWQRR"]
#14
#enzymes = ["MRIAMIAAMANNRVIGKDNQMPWHLPEDLKHFKAMTLGKPVVMGRKTFESIGRPLPGRHNIVISRQSDYSHEGISLVSSFEEAVALAGDCEELVVIGGGQLYQTLISQADKLYLTEIALEVEGDTHFPAWDDGSWETVESTEHVSDKGLQYRFITLVKKC"]
#16
#enzymes = ["MISMIWAMGRNNALGCKNRMPWYIPADFAYFKKVTMGKPVIMGRKTFESIGKPLPGRKNIVITRDTGYDPQGCIVVNSIEKAMEYTEEKEVFIIGGAEIYKEFLPIADRLYITLIEKEFEADAFFPEIDYSKWKQISCETGIKDEKNPYEYKWLVYERVKQ"]
#17
#enzymes = ["MTKEMIAIWAEDEAGLIGVDGKLPWYLPRELQHFKETTLNQAILMGRVTFDGMNRRLLPNRQTLVMTRDVNYQVDGVLTMTSVEEVLDWYHAQEKTLYVIGGSKVLEAFDGYFNRVIKTVVHHQFEGDTYRPKLDLSRFREESQTFYPKDANNPYDFTVTVLKHQ"]
#19
#enzymes = ["MINLIWAQDANGNIGKDNQMPWRLPADLAYFKKQTTGKTIVMGRKTYDSLGKALPNRENIVLTRDPELTLADASVVHTKADILKRAEKEAIFIIGGAQIYALFADDADRLYVTKIAATFDADTAFPVLDWSAFRLVQETPGHIDEKNKYPHAFLVYEREQEIQ"]
#20
#enzymes = ["MIISLIAALAADRVIGMENAMPWHLPADLAWFKRNTLNKPVIMGRKTFESIGRPLPGRLNIVISSQPGADSRVSWATSIDEALALAGEVEEVMVMGGGHVYSQFLDRAHRMYLTHIDAEVGGDTHFPDYEPDEWETSFSEFHDADEANSHSYCFEVLERR"]
#21
#enzymes = ["MHKMAEKNLIWAQTLDGTIAIDGHIPWHQKADLQFFKQSTIHEAALMGRHTMASFHGRPLPERLNLVLTRNHDLEVPEGFQKVYSVAEAEKVADNANLKLQVIGGKPIYESFMDTADTLYVTYLQTDFSGDVKMAPVDLTVWQGEVIDQGPADADNDYDWRLVKYTRQ"]
#22
#enzymes = ["MTTISLIAAIDESGGLGINNQLLCHLPADLQHFKSITMGKPIIMGRKTFDSIGKPLPGRLNIVLSRSLSFIDGVHVLDSLEKAIEYTAGHDEIMIIGGAVLFNDAIDKASSLYITHIHHQFAADVFFPKIDKSVWHCQNEQFRHQDEKNKHDMTFCFYERKH"]
#23
#enzymes = ["MKLSLMVAISKNGVIGNGPDIPWSAKGEQLLFKAITYNQWLLVGRKTFESMGALPNRKYAVVTRSSFTSDNENVLIFPSIKDALTNLKKITDHVIVSGGGEIYKSLIDQVDTLHISTIDIEPEGDVYFPEIPSNFRPVFTQDFASNINYSYQIWQKG"]
#26
#enzymes = ["MKLTLIVAFDRNNAIGRDNDLPWKLPDDLKRFKALTLGKPILMGRKTAQSLGRALPERLNLVLTRSGQVPFEGMHAVASVEQAIERAAQDGAQELCVIGGGEVYRLTMQRADLLAVTEVDTVVDGADTHFPPIDPALWEAVQRESHAADARHAFAFDYVDYRRR"]
#27
#enzymes = ["MNTSLPLSLIAAYAENRVIGIDNSMPWHLPGDFKYFKATTLGKPIIMGRKTWDSLGRPLPGRLNIVVSRQPGLELAGAEVFASLEEALVRAEQWAREQGVGELMLIGGAQLYGQALEKGLVSRMYLTRVELAPEGDAWFPAFDEGQWRLTSSEAQAEEGKPAYHFEVWDKG"]
#30
#enzymes = ["MIISFIVAMDENHLIGANRRLPWHLPADLKYFRRMTMGKPMLMGRSTHEAIGRPLPGRHNIVVTHNPDYKAPGCTVVHTVEAGLQAAGEAEEVTVIGGASLYEQLLPQARRIYLTLVRGHFQGDTWFPAFDPGTWVEVWREDYDPDPQNPYPYSFIRLEREERKK"]
#31
#enzymes = ["MISMIWAEDLRHGIGKQQSIPWHIPGDMAFFKKTTSGNTVVMGRKTFDSIGKALPNRKNIVLSHHPTSLPDSVVGVGSLSELQAIFETHPNEKFYIIGGSHLYNALLSQADELLITRIQKDYQCEVLAPDITQNEFKLASSVEHESTERNPAYTFETWIRI"]
#32
#enzymes = ["MISIIVAIAKDNVIGKDNKLLWHISEDLKRFKKITTGKKMIMGRKTFESLPGILPNREHIVLTRDNNFNVDSDKVTIEHDFNSVLQRYSECEDEVFVIGGAEIYKQFLPYAKKLYLTKVDEEFEGDTYFPGINYSNYNTEYTSEKFIDEKNGLHYTFVNLIQA"]
#33
#enzymes = ["MISLIAALAVDRVIGMENAMPWNLPADLAWFKRNTLNKPVVMGRHTWESIGRPLPGRKNIVISSQPGTDDRVQWVKSVDEAIAACGDAPEIMVIGGGRVYEQFLPKAQKLYLTHIDAEVEGDTHFPDYEPDDWESVFSEFHDADEQNSHSYCFEILERR"]
#35
#enzymes = ["MIISLIAALSADRVIGLDNAMPWHLSADLAWFKRNTLAKPVIMGRKTYESIGRPLPGRHNIVISRQAGCDDRVSWVATPEAALTAAADADEVMIIGGGTLYEQFLPQASRLYLTHVDADVAGDTYFPDYAPDEWQTCFSEFHEADSDNSHAYCFEILDRR"]
#36
#enzymes = ["MKIGLVAALTRNQVIGRYNALPWSLPADLQRFKKITMGKPIIMGRKTYDSIGRPLPGRKNIVISRNPDFHADGVTVVDSLDMALEAADQAPEVMVIGGANIYYQFLPRADRLYFTLVHTTIDDGDAFFPAYNRRDWRLVIEENHPADADNPFPYSFMTWQRITPPKNQSDTPEAHHI"]
#37
#enzymes = ["MAPVFFSEPFNMISLIAALAADRIIGMENAMPWDLPADLAWFKRNTLKKPVIMGRLTFESIGRPLPGRLNIVVSSKPGSAEGVTWVTSLDDAISAAGEAEEIMVIGGGRIYEQMLARADRLYLTHIDAEVEGDTQFPDYEPDEWHSTFSEFHDADEQNSHSYCFEILERRR"]
#38
#enzymes = ["MITSIWAQSKNNVIGRNNRLPWSLPDDLKFFRQETKNKAVVMGRKTYESFGSKALPKRLNIILTSNMDFKSNDPEVKIVHSPIEAVNTAKEQHLPLYVIGGASVYESFMNMADRLLITLVDANIKGDTFAPNFSEKSFSLVSQCHHAQDKKHAYSFDFLTYERKNKSDQV"]
#39
#enzymes = ["MTTLTLIVARARNGVIGRDNQLPWKLPEDLAFFKRTTMGAPIVMGRRTHESIGRPLPGRRNIVVTRDAARRFDGCDTVTSLDGALALAARDGAAEAFLIGGAQLYAEGLRHADKLIVTEIDQDFEGDASFPAPDPAQWEAVSRDAHRAAAPNDFAYAFVVYRRKRAG"]
#41
#enzymes = ["MPKITLIAACAPDRCIGINNTMPWHLPEDFAFFKSYTLDKPVVMGRKTWESLPRKPLPGRRNIVISRQADYPAEGAETVGSLEEALALCAAAEEVIIMGGAQIYAQALPLASDLRITEVALDVQGDAFFPEFSPSEWREAARETHTSANGTGYAFVHYTRIR"]
#42
#enzymes = ["MMISMIAAMAENRIIGKDNQMPWHLPADFAWFKRCTLGKPVIMGRKTYQSIGRPLPGRHNIVISRDESLQIDGVDVVTSIDAALAKAGEVDEIMIIGGGSLYAACLPMANKLYITEINAELDGDTQFPAWGSDWQECYREHYPADEKNTYAMDFVIFERK"]
#43
enzymes = ["MSEARPFTLALVVAAAENDVIGREGKLPWRLKSDLKRFRRLTMGHPLIMGRKTFASIGKPLDGRDSVIVTRDAASVSPQPGVFVATSIKDALDIARARAAERRVAEAFVIGGAEIFALALPYADRIHLARVHASPSGDAYWQAPPAEEWQVVSREEWPANQTDEFSVTDLVLERSPRA"]









In [4]:
df = kcat_predicton(substrates = substrates,
               products = products,
               enzymes = enzymes)

Step 1/3: Calculating numerical representations for all substrates and products.
Step 2/3: Calculating numerical representations for all enzymes.
.....2(a) Loading ESM-1b model.


[19:13:13] WARNING: not removing hydrogen atom without neighbors
[19:13:13] DEPRECATION WARNING: please use AtomPairGenerator
[19:13:13] DEPRECATION WARNING: please use AtomPairGenerator
[19:13:13] DEPRECATION WARNING: please use AtomPairGenerator
[19:13:13] DEPRECATION WARNING: please use AtomPairGenerator
[19:13:13] DEPRECATION WARNING: please use AtomPairGenerator


.....2(b) Loading model parameters for task-specific model.
.....2(c) Calculating enzyme representations.
Step 3/3: Making predictions for kcat.


In [5]:
df

,substrates,products,enzyme,difference_fp,enzyme rep,complete,kcat [s^(-1)]
0,InChI=1S/C19H23N7O6/c20-19-25-15-14(17(30)26-1...,InChI=1S/C19H21N7O6/c20-19-25-15-14(17(30)26-1...,MSEARPFTLALVVAAAENDVIGREGKLPWRLKSDLKRFRRLTMGHP...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.097802, 0.6335388, -0.16979533, 0.9875663, ...",True,20.686501


In [ ]:
pred = [17.18,
14.7,
14.5,
15.7,
12.4,
18.36,
12.85,
11.26,
21.04,
21.04,
9.2,
20.88,
37.24,
20.53,
30.31,
7.43,
18.92,
15.25,
15.64,
25.97,
14.47,
18.59,
19.25,
15.3,
10.12,
5.26,
25.15,
20.43,
14.47,
12.56,
31.23,
24,
20.69]

true = [30.8,
4.1,
44.2,
10.9,
17.8,
16.3,
4.4,
3.9,
15.3,
34.6,
28.1,
35.2,
28.1,
35.2,
13.8,
20,
31.4,
19.5,
3.9,
4.6,
14.3,
10,
48.9,
71.7,
1.5,
8.2,
14.7,
3.7,
25.9,
8.1,
53.7,
14.6,
8.1]

from sklearn.metrics import root_mean_squared_error

print(root_mean_squared_error(true, pred))  #16.65

colors = np.arange(0,32,1)



In [ ]:
print(len(true))

import matplotlib.pyplot as plt


x = np.arange(0,33,1)
print(len(x))

plt.scatter(x, true, color = 'red')
plt.scatter(x, pred, color = 'blue')



#plt.scatter(true, pred)